In [44]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site


from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

USER = os.environ["USER"]

MAX_THRESHOLD_CONST = 2.34 #CONSTANT WILL BE mean of rats for stim max / threshold for EVERY possible combination... ma.mean(ma.median((stim_max/ma.mean(a, axis=0)),axis = (1,2,3)),axis =0)
NUM_THRESHOLD_POINTS = 500

In [2]:
src = f"/home/andres/repos/rat-mapping-paper/reports/J_SHAP/non_hierarchical_bayesian_model/inference.pkl"
with open(src, "rb") as f:
    (
        df,
        encoder_dict,
        model,
        posterior_samples,
    ) = pickle.load(f)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
a = posterior_samples[site.a]
b = posterior_samples[site.b]
L = posterior_samples[site.L]
ell = posterior_samples[site.ell]
H = posterior_samples[site.H]

x = np.linspace(0, 500, 1000)

named_params = [site.a, site.b, site.L, site.ell, site.H]
params = [posterior_samples[param][ ...] for param in named_params]

for named_param, param in zip(named_params, params):
    
    print(named_param, param.shape)

a (4000, 8, 16, 4, 6)
b (4000, 8, 16, 4, 6)
L (4000, 8, 16, 4, 6)
ℓ (4000, 8, 16, 4, 6)
H (4000, 8, 16, 4, 6)


In [4]:
subjects = sorted(df['participant'].unique())
# subjects = encoder_dict['participant'].inverse_transform(subjects)
positions = sorted(df['compound_position'].unique())
# positions = encoder_dict['compound_position'].inverse_transform(positions)
charges = sorted(df['compound_charge_params'].unique())
# charges = encoder_dict['compound_charge_params'].inverse_transform(charges)
muscles = model.response


In [7]:
compound_position = encoder_dict[model.features[1]].inverse_transform(sorted(df['compound_position'].unique()))
mono = [c for c in compound_position if c.split("-")[0] == "" or c.split("-")[1] == ""]
mono = np.array(mono)
mono = mono
mono_inds = encoder_dict[model.features[1]].transform(mono)

In [76]:
compound_position

array(['-C6L', '-C7L', '-C8L', 'C6L-', 'C6L-C6M', 'C6L-C7L', 'C6M-C6L',
       'C7L-', 'C7L-C6L', 'C7L-C7M', 'C7L-C8L', 'C7M-C7L', 'C8L-',
       'C8L-C7L', 'C8L-C8M', 'C8M-C8L'], dtype=object)

In [8]:
mono_inds

array([ 0,  1,  2,  3,  7, 12])

In [11]:
cathode = [0,1,2] 
anode = [3,7,12]
mono_inds = [(0,1,2), (3,7,12)]

In [16]:
electrodes = ['cathode', 'anode']

In [41]:
stim_max = []

for s in subjects:
    for e_ind, e in enumerate(electrodes):
        for p in mono_inds[e_ind]:
            for m in muscles: 
                temp = df[(df['participant'] == s) 
                        & (df['compound_position'] == p) 
                        & (df['compound_charge_params'] == 1)
                        ]
                temp = temp.loc[:,[m, 'pulse_amplitude']]
                temp = temp.pulse_amplitude.max()
                stim_max.append(temp)
stim_max = ma.array(stim_max)
stim_max = stim_max.reshape(len(subjects), len(electrodes), len(mono_inds[0]), len(muscles))

In [42]:
stim_max.shape

(8, 2, 3, 6)

In [21]:
a.shape

(4000, 8, 16, 4, 6)

In [36]:
encoder_dict['compound_charge_params'].inverse_transform([0,1,2,3])

array(['20-0-80-25', '50-0-50-0', '50-0-50-100', '80-0-20-400'],
      dtype=object)

In [39]:
arr = []
for s in subjects:
    for e_ind, e in enumerate(electrodes):
        for p in mono_inds[e_ind]:
            a_temp = a[:, s, p, 1,:]
            arr.append(a_temp)
arr = np.array(arr)
arr = arr.reshape(len(subjects),len(electrodes), len(mono_inds[0]), *arr.shape[1:])

In [40]:
arr.shape

(8, 2, 3, 4000, 6)

In [43]:
ma.mean(ma.median((stim_max/ma.mean(arr, axis=3)),axis = (1,2,3)),axis = 0)

2.3445656142722955

In [29]:
arr.shape

(8, 2, 3, 4000, 4, 6)

In [38]:
mono_inds = [(0,1,2),(3,7,12)]

In [49]:
mono_inds

[(0, 1, 2), (3, 7, 12)]

In [46]:
norm_y = []
norm_x = []

for subject_ind, subject in enumerate(subjects):
        # Fix the subject index and size index
    curr_params = [
        params[:, subject_ind, mono_inds, 1,:, None] for params in params
    ]

    # Take nanmean across posterior samples
    temp = np.nanmean(curr_params[0], axis=0)
    
    # Take nanmedian across compound positions and muscles
    median_threshold = np.nanmedian(temp)

    x_temp = np.linspace(0.,  median_threshold * MAX_THRESHOLD_CONST, NUM_THRESHOLD_POINTS)
    x_temp = x_temp[ None, None, None,:]

    temp_thresh = F.rectified_logistic(
        x_temp, #x_temp
        *curr_params
    )
    # Remove offset
    temp_thresh = temp_thresh - curr_params[2]
    norm_y.append(temp_thresh)
    norm_x.append(x_temp)
norm_x = np.array(norm_x)
norm_x = norm_x.reshape(len(subjects),  *norm_x.shape[0:])
norm_y = np.array(norm_y)
norm_y = ma.masked_invalid(norm_y)
norm_y = norm_y.reshape(len(subjects),  *norm_y.shape[0:])

/tmp/ipykernel_3692217/3273236.py:11: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(curr_params[0], axis=0)


In [50]:
norm_y.shape

(8, 4000, 2, 3, 6, 500)

In [52]:
cath_y = norm_y[:,:,0,...]
anode_y = norm_y[:,:,1,...]

In [54]:
cath_y.shape

(8, 4000, 3, 6, 500)

In [58]:
cath_y_max = ma.max(cath_y, axis = (2,3,4), keepdims=True)
anode_y_max = ma.max(anode_y, axis = (2,3,4), keepdims=True)

In [59]:
cath_y = ma.where(cath_y_max, cath_y / cath_y_max, 0.)
anode_y = ma.where(anode_y_max, anode_y / anode_y_max, 0.)

In [60]:
cath_y.shape

(8, 4000, 3, 6, 500)

In [62]:
cath_p = ma.sum(cath_y, axis=-2, keepdims=True)
anode_p = ma.sum(anode_y, axis=-2, keepdims=True)
cath_p = ma.where(cath_p, cath_y / cath_p, cath_y.shape[-2])
anode_p = ma.where(anode_p, anode_y / anode_p, anode_y.shape[-2])

In [63]:
plogp_cath = ma.where(cath_p, cath_p * ma.log(cath_p), 0.)
plogp_anode = ma.where(anode_p, anode_p * ma.log(anode_p), 0.)

In [64]:
cath_entropy = 1 + (plogp_cath.sum(axis=-2) / ma.log(cath_y.shape[-2]))
anode_entropy = 1 + (plogp_anode.sum(axis=-2) / ma.log(anode_y.shape[-2]))

In [65]:
auc_cath = np.trapz(y=cath_entropy[...], x=np.linspace(0, MAX_THRESHOLD_CONST, NUM_THRESHOLD_POINTS), axis=-1)
auc_anode = np.trapz(y=anode_entropy[...], x=np.linspace(0, MAX_THRESHOLD_CONST, NUM_THRESHOLD_POINTS), axis=-1)

In [67]:
auc_cath = ma.mean(auc_cath, axis=1)
auc_anode = ma.mean(auc_anode, axis=1)

In [74]:
ma.mean(auc_cath, axis=(0,1))


14.97841301829909

In [75]:
ma.mean(auc_anode, axis=(0,1))


31.23043700542463

In [68]:
mat = auc_anode - auc_cath

In [70]:
mat = ma.mean(mat, axis=1)

In [71]:
stats.wilcoxon(mat)

WilcoxonResult(statistic=5.0, pvalue=0.078125)

In [40]:
compound_position = encoder_dict[model.features[1]].inverse_transform(sorted(df['compound_position'].unique()))
mono = [c for c in compound_position if c.split("-")[0] == "" or c.split("-")[1] == ""]
mono = np.array(mono)

In [8]:
model.features[1]

'compound_position'

In [42]:
mono

array(['-C6L', '-C7L', '-C8L', 'C6L-', 'C7L-', 'C8L-'], dtype='<U4')

In [43]:
mono = encoder_dict[model.features[1]].transform(mono)


In [ ]:
compound_charge = encoder_dict[model.features[2]].inverse_transform(sorted(df['compound_charge_params'].unique()))
print(compound_charge)

['20-0-80-25' '50-0-50-0' '50-0-50-100' '80-0-20-400']


In [21]:
ind = [0,1,2]

In [32]:
mono[:3]

array([0, 1, 2])

In [24]:
arr = arr.swapaxes(0, 1)


In [26]:
arr = arr[:, :, ind, ...]

In [57]:
arr = norm_y
arr = ma.masked_invalid(arr)
arr.shape

(8, 4000, 16, 4, 6, 500)

In [47]:
test = norm_y[...,mono,1,:,:]
test.shape

(8, 4000, 6, 6, 500)

In [58]:
def get_entropy(elec_ind):
    if elec_ind == 0:
        elec_ind = mono[:3]
    else:
        elec_ind = mono[3:]
    y = arr[...,elec_ind,1,:,:]
    x = norm_x

    y_max = ma.max(y, axis=(2,3, -1), keepdims=True) 

    y = ma.where(y, y / y_max, 0.)

    p = ma.sum(y, axis=-2, keepdims=True)
    p = ma.where(y, y / p, 0.)

    plogp = ma.where(p, p * ma.log(p), 0.)

    entropy = ma.where(
        ma.any(p, axis=-2, keepdims=True),
        (
            1
            + (ma.sum(plogp, axis=-2, keepdims=True) / np.log(plogp.shape[-2]))
        ),
        0.
    )
    entropy = entropy[..., 0, :]
    entropy.shape

    return entropy, x

In [59]:
cathodic_entropy, cathodic_x = get_entropy(0)
anodic_entropy, anodic_x = get_entropy(1)

In [35]:
pseudo25_entropy, pseudo25_x = get_entropy(0)
mono_entropy, mono_x = get_entropy(1)
bi_entropy, bi_x = get_entropy(2)
pseudo80_entropy, pseudo80_x = get_entropy(3)


In [52]:
def AUC(entropy, x):
    area_list = []
    for subject_ind, subject in enumerate(subjects):
        x_temp = x[subject_ind, 0,0, ...]
        sample_integrals = entropy[:, subject_ind, ...]
        area = np.trapz(sample_integrals, x_temp)
        area_list.append(area)

    area_list = np.array(area_list)
    area_list = area_list.reshape(len(subjects), *area_list.shape[1:])
    area_list = ma.masked_values(area_list, 0)
    area_list_mean = ma.mean(area_list, axis=(-1, -2))
    
    return area_list_mean
    

In [60]:
cathodic = AUC(cathodic_entropy, cathodic_x)
anodic = AUC(anodic_entropy, anodic_x)

In [48]:
pseudo25 = AUC(pseudo25_entropy, pseudo25_x)
mono = AUC(mono_entropy, mono_x)
bi = AUC(bi_entropy, bi_x)
pseudo80 = AUC(pseudo80_entropy, pseudo80_x)

In [54]:
auc_list = [cathodic , anodic]

In [61]:
anodic

masked_array(data=[401.8133227804154, 498.29304357476747,
                   526.457555664568, 216.19598743128378,
                   105.27595208439043, 199.48292263557556,
                   114.48068324463604, 58.940062908339925],
             mask=[False, False, False, False, False, False, False, False],
       fill_value=1e+20)

In [62]:
src = f"/home/andres/repos/rat-mapping-paper/notebooks/J_SHAP/cath_anode_selectivity_means.pkl"

with open(src, "wb") as f:
    pickle.dump([cathodic , anodic], f)

In [68]:
for a1_ind, auc1 in enumerate(auc_list):
    for a2_ind, auc2 in enumerate(auc_list):
        if a1_ind == a2_ind:
            continue
        t = auc1 - auc2
        ttest = stats.ttest_1samp(t, popmean=0, axis=0, alternative='less')
        pvalue = ttest.pvalue
        print(compound_charge[a1_ind], compound_charge[a2_ind], pvalue)

20-0-80-25 50-0-50-0 0.13942974345812995
20-0-80-25 50-0-50-100 0.04215669430193781
20-0-80-25 80-0-20-400 0.02894819264413264
50-0-50-0 20-0-80-25 0.86057025654187
50-0-50-0 50-0-50-100 0.20342394860276447
50-0-50-0 80-0-20-400 0.03345596220632771
50-0-50-100 20-0-80-25 0.9578433056980622
50-0-50-100 50-0-50-0 0.7965760513972355
50-0-50-100 80-0-20-400 0.041965577763092204
80-0-20-400 20-0-80-25 0.9710518073558674
80-0-20-400 50-0-50-0 0.9665440377936723
80-0-20-400 50-0-50-100 0.9580344222369078
